In [4]:
import pandas as pd
import time

import requests
from bs4 import BeautifulSoup as bs
import re
from selenium.webdriver.common.action_chains import ActionChains
import os

# selenium==4.2.0 
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service


# Get report & county information
save those infor to pickle

In [ ]:
# get all county
county_url='https://dq.cde.ca.gov/dataquest/page2.asp?level=County&subject=Enrollment&submit1=Submit'
response = requests.get(county_url)
soup = bs(response.text, 'html.parser')
cCountys=[]
for option in soup.find('select', {'name': 'cCounty'}).find_all('option'):
    text = option.text.strip().replace(' ','+')
    cCountys.append(text)


# step 1: get all 'Subject & topics' options (they are the same)
require_years=["2022-23","2021-22","2020-21","2019-20","2018-19","2017-18"]  #
level='County'

url = 'https://dq.cde.ca.gov/dataquest/'
response = requests.get(url)
soup = bs(response.text, 'html.parser')
options = []
for select in soup.find_all('select'):
    for option in select.find_all('option'):
        label = option.get('label')
        content=option.text.strip()
        subject = option.get('value')
        try:
            step2='https://dq.cde.ca.gov/dataquest/page2.asp?level={}&subject={}&submit1=Submit'.format(level,subject)
            soup2 = bs(requests.get(step2).text, 'html.parser')
            rYears=[]
            for option in soup2.find('select', {'name': 'rYear'}).find_all('option'):
                text = option.get('value')
                rYears.append(text)
                
            intersection_set = set(require_years) & set(rYears)
            rYears = list(intersection_set)
            rYears.sort(reverse=True)
        except:
            rYears=''
        
        options.append({'label': label,'content': content, 'subject': subject,'exist_year':rYears})
df1 = pd.DataFrame(options)
df1 = df1.loc[df1['exist_year'].apply(lambda x: len(x) > 0),:]

#df1.to_csv('CA_data/base_information.csv')

    
# save data
import pickle
data = (df1, cCountys)
bytes_data = pickle.dumps(data)
with open("CA_data/variables3.pickle", "wb") as f:
    f.write(bytes_data)

In [ ]:
df1[1:]

,label,content,subject,exist_year
13,Annual Enrollment Data,Annual Enrollment Data,Enrollment,"[2021-22, 2020-21, 2019-20, 2018-19, 2017-18]"
14,English Learner Data,English Learner Data,LC,"[2021-22, 2020-21, 2019-20, 2018-19, 2017-18]"
15,Foster Student Data,Foster Student Data,Foster,"[2020-21, 2019-20, 2018-19, 2017-18]"
16,Special Education Data,Special Education Data,SpecEd,"[2018-19, 2017-18]"
17,Four-Year Cohort Graduation Rates & Outcomes,Four-Year Cohort Graduation Rates & Outcomes,Coh,"[2021-22, 2020-21, 2019-20, 2018-19, 2017-18]"
18,Five-Year Cohort Graduation Rates,Five-Year Cohort Graduation Rates,Coh5,"[2021-22, 2020-21, 2019-20, 2018-19, 2017-18]"
21,College-Going Rates,College-Going Rates,CGR,"[2019-20, 2018-19, 2017-18]"
22,Suspension and Expulsion Data,Suspension and Expulsion Data,Expulsion,"[2021-22, 2020-21, 2019-20, 2018-19, 2017-18]"
23,Absenteeism Data,Absenteeism Data,Attendance,"[2021-22, 2020-21, 2019-20, 2018-19, 2017-18]"
24,Stability Rates,Stability Rates,STB,"[2021-22, 2020-21, 2019-20, 2018-19, 2017-18]"


# Web scarping
## type1:   [ 'SpecEd' ,'Hires','FRPM','Foster']   √

[ 'SpecEd' ,'Hires']:
- no filter option(no 'non char shchool').
- no subtable, no need to open `<a>`
 
1. 'SpecEd' can be directly saved as HTML and opened with CSV.
2. 'Hires'&'FRPM' can define sheetname directly
3. Fprm: 'Free and Reduced Price Meal - County summary (with district data)'. only one report is enough
   1. 'FRPM' can be accessed without the need for the original loop, as the year can be changed directly on the webpage. However, this method may be too cumbersome and it is better to include it in the original loop.
4. 'Foster'

In [3]:
# Load the variables from the file
import pickle
with open("CA_data/variables.pickle", "rb") as f:
    df1,cCountys = pickle.load(f)

### 'SpecEd'

In [36]:
chrome_options = Options()
chrome_options.add_argument('--headless')
driver = webdriver.Chrome(executable_path='/Users/dai/Downloads/chromedriver_mac_arm64/chromedriver', options=chrome_options)

def download_ca_SpecEd(step3,subject,year,cCounty):
    driver.get(step3)
    
    report_select =  driver.find_elements(by=By.XPATH, value='//input[@name="cChoice"]')
    m=[report_select[i].get_attribute('value') for i in range(len(report_select))]

    for option in m:
        option_element = driver.find_element(by=By.XPATH, value=f"//input[@value='{option}']")
        option_element.click()

        submit_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//input[@value='Submit']"))
        )
        submit_button.click()
        
        # Step4: Get into the last website
        folder_path=os.path.join('/Users/dai/Desktop/Capstone-AllianceBernstein/CA_data/',subject,option,year)
        if not os.path.exists(folder_path):
            os.makedirs(folder_path)
        
        # Write the table to an Excel file
        df = pd.read_html(driver.page_source)
        df[0].to_excel(os.path.join(folder_path,cCounty+'.xlsx'), index=False)

        with open(os.path.join(folder_path,cCounty+'.html'), 'w') as f:
            f.write(driver.page_source)
        
        driver.back()

In [ ]:
driver = webdriver.Chrome(executable_path='/Users/dai/Downloads/chromedriver_mac_arm64/chromedriver')
subject='SpecEd'
year='2018-19'
cCounty='01+ALAMEDA'
level='County'

if subject=='SpecEd':
    step3='https://dq.cde.ca.gov/dataquest/SearchName.asp?rbTimeFrame=oneyear&rYear={}&cCounty={}&Topic={}&Level={}&submit1=Submit'.format(year,cCounty,subject,level) 
    #step3='https://dq.cde.ca.gov/dataquest/SearchName.asp?rbTimeFrame=oneyear&rYear=2018-19&cCounty=01+ALAMEDA&Topic=SpecEd&Level=County&submit1=Submit'
    download_ca_SpecEd(step3,subject,year,cCounty)

### 'Hires' & 'FRPM' & 'foster'
no char school option

In [18]:
def download_ca_s4(step4,subject,cCounty,year,report=None): 
    driver.get(step4)

    # Step4: Get into the last website  # find all table in html
    tables = driver.find_elements(by=By.TAG_NAME, value="table")

    # Store each table and its title in a different DataFrame
    dfs = []
    for table in tables:  
        # covert html table to DataFrame
        df = pd.read_html(table.get_attribute("outerHTML"), header=0)[0]
        # save dataframe into list
        if not df.empty:
            dfs.append(df)
            
    folder_path=os.path.join('/Users/dai/Desktop/Capstone-AllianceBernstein/CA_data/',subject)  
    if report:
        folder_path=os.path.join(folder_path,report)
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
    filepath=os.path.join(folder_path,,year,cCounty+'.xlsx')
    
    sheet_names=['Sheet{}'.format(i+1)  for i in range(len(dfs))]
    if report and len(dfs)>=2:
        sheet_names[-2:]=[report,'ReportTotals']

    writer = pd.ExcelWriter(filepath, engine='xlsxwriter')  
    for i, df in enumerate(dfs):
        df.to_excel(writer, index=False,sheet_name=sheet_names[i])
    writer.save()

    # Make sure all downaload have been finished
    time.sleep(2) 
   

In [ ]:
year='2018-19'
cCounty='01+ALAMEDA'
cds=cCounty.split('+')[0]
level='County'

#driver = webdriver.Chrome(executable_path='/Users/dai/Downloads/chromedriver_mac_arm64/chromedriver')

subject='Foster'
subject='Paif'
subject='FPRM'
subject='Hires'

if subject=='Foster':
    reports=['fosterGrdEnrl','fosterGrdRace']
    for report in reports:
        step4='https://dq.cde.ca.gov/dataquest/{}/{}.aspx?level={}&cds={}&year={}'.format(subject,report,level,cds,year)
        #https://dq.cde.ca.gov/dataquest/foster/fosterGrdEnrl.aspx?level=County&cds=01&year=2020-21
        #https://dq.cde.ca.gov/dataquest/foster/fosterGrdRace.aspx?level=County&county=01&year=2020-21
        download_ca_s4(step4,subject,cCounty,year,report)
        
if subject =='Paif':
    reports=['StfFteClassified','StfFteClassifiedLevels']
    for report in reports:
        step4='https://dq.cde.ca.gov/dataquest/dqcensus/{}.aspx?cds={}&agglevel={}&year={}'.format(report,cds,level,year)
        #https://dq.cde.ca.gov/dataquest/dqcensus/StfFteClassified.aspx?cds=01&agglevel=County&year=2021-22
        #https://dq.cde.ca.gov/dataquest/dqcensus/StfFteClassifiedLevels.aspx?cds=01&agglevel=County&year=2021-22
        download_ca_s4(step4,subject,cCounty,year,report)

if subject=='FPRM':
    step4='https://dq.cde.ca.gov/dataquest/Cbeds2.asp?FreeLunch=on&cChoice=CoProf2&cYear={}&TheCounty={}&cLevel=County&cTopic={}&myTimeFrame=S&submit1=Submit'.format(year,cCounty,subject)
    #https://dq.cde.ca.gov/dataquest/Cbeds2.asp?FreeLunch=on&cChoice=CoProf2&cYear=2021-22&TheCounty=01%2CALAMEDA&cLevel=County&cTopic=FRPM&myTimeFrame=S&submit1=Submit
    download_ca_s4(step4,subject,cCounty,year,report=None)

if subject=='Hires':
    #https://dq.cde.ca.gov/dataquest/dqcensus/StfTchHires.aspx?cdcode=01&agglevel=County&year=2022-23
    step4='https://dq.cde.ca.gov/dataquest/dqcensus/StfTchHires.aspx?cdcode={}&agglevel={}&year={}'.format(cds,level,year)
    download_ca_s4(step4,subject,cCounty,year,report=None)

## Rest reports

In [25]:
def table_download(driver,filepath):
    tables = driver.find_elements(by=By.TAG_NAME, value="table")
    dfs = []
    for table in tables:  
        df = pd.read_html(table.get_attribute("outerHTML"), header=0)[0]
        if not df.empty:
            dfs.append(df)
    
    h_tags = driver.find_elements(by=By.TAG_NAME, value='h1') + \
                driver.find_elements(by=By.TAG_NAME, value='h2') 
                
    header=[]
    for h in h_tags:
        header.append(h.text) 
    header=[i for i in header if i != '']

    try:
        if len(header[0]+'-'+header[1])>31:
            sheet_names=[(header[0]+'-'+header[1]).replace(' ','')[0:31]]
            sheet_names.extend(header[2:])
        else:
            sheet_names=[header[0]+'-'+header[1]]
            sheet_names.extend(header[2:])
            
        if len(sheet_names) != len(dfs):
            #for i in range(abs(len(sheet_names)-len(dfs))):
            for i in range(len(sheet_names), len(dfs)):
                sheet_name = f"sheet{i+1}"
                sheet_names.insert(0, sheet_name)
    except:
        sheet_names=['Sheet{}'.format(i+1)  for i in range(len(dfs))]
                            
    writer = pd.ExcelWriter(filepath, engine='xlsxwriter')  
    for i, df in enumerate(dfs):
        df.to_excel(writer, index=False,sheet_name=sheet_names[i])
    writer.save()

def download_ca_t2(step3,subject,cCounty,year): 
    driver.get(step3)

    report_select =  driver.find_elements(by=By.XPATH, value='//input[@name="cChoice"]')
    m=[report_select[i].get_attribute('value') for i in range(len(report_select))]

    form = driver.find_element(by=By.TAG_NAME, value="form")
    text = driver.execute_script("return arguments[0].textContent;", form)
    text=[t.strip('\t') for t in text.split('\n') if t.strip()]  
    text=[s.strip().replace('\xa0', '') for s in text]
    text=text[-len(m):]
    text=[i.lower() for i in text]

    dict_option={key: value for key, value in zip( m,text)}
    values_list = list(dict_option.values())
    for option, value in dict_option.items():
        if value+' (with district data)' in values_list or value+' (with dist. data)' in values_list:
            pass
        else:  # web scarping
            folder_path=os.path.join('/Users/dai/Desktop/Capstone-AllianceBernstein/CA_data/',subject,option,year)
            if not os.path.exists(folder_path):
                os.makedirs(folder_path)
                
            option_element = driver.find_element(by=By.XPATH, value=f"//input[@value='{option}']")
            option_element.click()

            submit_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, "//input[@value='Submit']"))
            )
            submit_button.click()
            
            a_tags = driver.find_elements(by=By.XPATH, value='//a[@data-toggle="collapse"]')
            for a in a_tags:
                ActionChains(driver).move_to_element(a).click().perform()
                time.sleep(1.5)

            # check if there are school filter option
            if driver.find_elements(by=By.XPATH, value="//*[contains(text(), 'School Type')]") and driver.find_elements(by=By.XPATH, value="//*[contains(text(), 'Charter')]"):
                
                # non-char school
                nonchar_button = WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.XPATH, '//input[@value="Y"]'))
                )
                nonchar_button.click()
                wait = WebDriverWait(driver, 4)
                filepath_nochar=os.path.join(folder_path,cCounty+'_no_char.xlsx')
                table_download(driver,filepath_nochar)
                
                # char shool
                char_button = WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.XPATH, '//input[@value="N"]'))
                )
                char_button.click()
                filepath_char=os.path.join(folder_path,cCounty+'_char.xlsx') 
                table_download(driver,filepath_char)
                
                # download individual charter school data 
                tables = driver.find_elements(by=By.TAG_NAME, value="table")
                href_list=[]
                for table in tables:  
                    a_list = table.find_elements(by=By.XPATH, value='.//a')
                    href_list.append([a.get_attribute('href') for a in a_list])
                #href_list=max(href_list[1:-1], key=len)
                href_list = [sublist for sublist in href_list if not any('https://www.cde.ca.gov/' in element or 'agglevel=State' in element for element in sublist)]
                href_list = [sublist for sublist in href_list if any('agglevel=school' in element for element in sublist)]
                href_list = [item for sublist in href_list if sublist for item in sublist if item is not None]

                #print(driver.current_url) #
                if len(href_list)>0:
                    for charschool in href_list:
                        download_ca_s4(charschool,subject,cCounty,year,report=None,char=True,option=option)
            else:
                filepath=os.path.join(folder_path,cCounty+'.xlsx') # no_char
                table_download(driver,filepath)

        time.sleep(3)
        driver.get(step3)
            
    # Make sure all downaload have been finished
    time.sleep(3) 

In [ ]:
level='County'
for ind in df1.iloc[[1]+[2]+list(range(5,12)),:].index:
    subject=df1.loc[ind,'subject']
    #Topic=subject
    years=df1.loc[ind,'exist_year']
    
    for year in years:
        for cCounty in cCountys:
            step3='https://dq.cde.ca.gov/dataquest/SearchName.asp?rbTimeFrame=oneyear&rYear={}&cCounty={}&Topic={}&Level={}&submit1=Submit'.format(year,cCounty,subject,level) 
            try:
                download_ca_t2(step3,subject,cCounty,year)
                time.sleep(4)
            except:
                break

# All Web scraping function above

In [23]:
import pandas as pd
import time
import requests
from bs4 import BeautifulSoup as bs
import re
from selenium.webdriver.common.action_chains import ActionChains
import os

# selenium==4.2.0 
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import TimeoutException

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

# Load the variables from the file
import pickle
with open("CA_data/variables.pickle", "rb") as f:
    df1,cCountys = pickle.load(f)

def download_ca_SpecEd(step3,subject,year,cCounty):
    driver.get(step3)
    
    report_select =  driver.find_elements(by=By.XPATH, value='//input[@name="cChoice"]')
    m=[report_select[i].get_attribute('value') for i in range(len(report_select))]

    for option in m:
        option_element = driver.find_element(by=By.XPATH, value=f"//input[@value='{option}']")
        option_element.click()

        submit_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//input[@value='Submit']"))
        )
        submit_button.click()
        
        # Step4: Get into the last website
        folder_path=os.path.join(os.getcwd()+'/CA_data/',subject,option,year)
        if not os.path.exists(folder_path):
            os.makedirs(folder_path)
        
        # Write the table to an Excel file
        df = pd.read_html(driver.page_source)
        df[0].to_excel(os.path.join(folder_path,cCounty+'.xlsx'), index=False)

        with open(os.path.join(folder_path,cCounty+'.html'), 'w') as f:
            f.write(driver.page_source)
        
        driver.back()
        
def download_ca_s4(step4,subject,cCounty,year,report=None): 
    driver.get(step4)

    # Step4: Get all tables in this webiste
    tables = driver.find_elements(by=By.TAG_NAME, value="table")

    # save each tables and its title to different DataFrame
    dfs = []
    for table in tables:  
        # covert table to DataFrame
        df = pd.read_html(table.get_attribute("outerHTML"), header=0)[0]
        # add DataFrame to list dfs.
        if not df.empty:
            dfs.append(df)
            
    folder_path=os.path.join( os.getcwd()+'/CA_data/',subject)  
    if report:
        folder_path=os.path.join(folder_path,report)
        
    folder_path=os.path.join(folder_path,year)
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)

    filepath=os.path.join(folder_path,cCounty+'.xlsx')
    
    sheet_names=['Sheet{}'.format(i+1)  for i in range(len(dfs))]
    if report and len(dfs)>=2:
        sheet_names[-2:]=[report,'ReportTotals']

    writer = pd.ExcelWriter(filepath, engine='xlsxwriter')  
    for i, df in enumerate(dfs):
        df.to_excel(writer, index=False,sheet_name=sheet_names[i])
    writer.save()

    # Make sure all downaload have been finished
    time.sleep(2) 


def table_download(driver,filepath):
    tables = driver.find_elements(by=By.TAG_NAME, value="table")
    dfs = []
    for table in tables:  
        df = pd.read_html(table.get_attribute("outerHTML"), header=0)[0]
        if not df.empty:
            dfs.append(df)
    
    h_tags = driver.find_elements(by=By.TAG_NAME, value='h1') + \
                driver.find_elements(by=By.TAG_NAME, value='h2') 
                
    header=[]
    for h in h_tags:
        header.append(h.text) 
    header=[i for i in header if i != '']

    try:
        if len(header[0]+'-'+header[1])>31:
            sheet_names=[(header[0]+'-'+header[1]).replace(' ','')[0:31]]
            sheet_names.extend(header[2:])
        else:
            sheet_names=[header[0]+'-'+header[1]]
            sheet_names.extend(header[2:])
            
        if len(sheet_names) != len(dfs):
            #for i in range(abs(len(sheet_names)-len(dfs))):
            for i in range(len(sheet_names), len(dfs)):
                sheet_name = f"sheet{i+1}"
                sheet_names.insert(0, sheet_name)
    except:
        sheet_names=['Sheet{}'.format(i+1)  for i in range(len(dfs))]
                            
    writer = pd.ExcelWriter(filepath, engine='xlsxwriter')  
    for i, df in enumerate(dfs):
        df.to_excel(writer, index=False,sheet_name=sheet_names[i])
    writer.save()

def download_ca_t2(step3,subject,cCounty,year): 
    driver.get(step3)

    report_select =  driver.find_elements(by=By.XPATH, value='//input[@name="cChoice"]')
    m=[report_select[i].get_attribute('value') for i in range(len(report_select))]

    form = driver.find_element(by=By.TAG_NAME, value="form")
    text = driver.execute_script("return arguments[0].textContent;", form)
    text=[t.strip('\t') for t in text.split('\n') if t.strip()]  
    text=[s.strip().replace('\xa0', '') for s in text]
    text=text[-len(m):]
    text=[i.lower() for i in text]

    dict_option={key: value for key, value in zip( m,text)}
    values_list = list(dict_option.values())
    for option, value in dict_option.items():
        if value+' (with district data)' in values_list or value+' (with dist. data)' in values_list:
            pass
        else:  # web scarping
            folder_path=os.path.join( os.getcwd()+'/CA_data/',subject,option,year)
            if not os.path.exists(folder_path):
                os.makedirs(folder_path)
            #print(folder_path) #
                
            option_element = driver.find_element(by=By.XPATH, value=f"//input[@value='{option}']")
            option_element.click()

            submit_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, "//input[@value='Submit']"))
            )
            submit_button.click()
            
            a_tags = driver.find_elements(by=By.XPATH, value='//a[@data-toggle="collapse"]')
            for a in a_tags:
                ActionChains(driver).move_to_element(a).click().perform()
                time.sleep(2)

            # check if there are school filter option
            if driver.find_elements(by=By.XPATH, value="//*[contains(text(), 'School Type')]") and driver.find_elements(by=By.XPATH, value="//*[contains(text(), 'Charter')]"):
                
                # non-char school
                try:
                    nonchar_button = WebDriverWait(driver, 20).until(
                        EC.element_to_be_clickable((By.XPATH, '//input[@value="Y"]'))
                    )
                    nonchar_button.click()
                    wait = WebDriverWait(driver, 5)
                    filepath_nochar=os.path.join(folder_path,cCounty+'_no_char.xlsx')
                    table_download(driver,filepath_nochar)
                    
                    # char shool
                    char_button = WebDriverWait(driver, 15).until(
                        EC.element_to_be_clickable((By.XPATH, '//input[@value="N"]'))
                    )
                    char_button.click()
                    filepath_char=os.path.join(folder_path,cCounty+'_char.xlsx') 
                    table_download(driver,filepath_char)
                    
                    # download individual charter school data 
                    tables = driver.find_elements(by=By.TAG_NAME, value="table")
                    href_list=[]
                    for table in tables:  
                        a_list = table.find_elements(by=By.XPATH, value='.//a')
                        href_list.append([a.get_attribute('href') for a in a_list])
                    #href_list=max(href_list[1:-1], key=len)
                    href_list = [sublist for sublist in href_list if not any('https://www.cde.ca.gov/' in element or 'agglevel=State' in element for element in sublist)]
                    href_list = [sublist for sublist in href_list if any('agglevel=school' in element for element in sublist)]
                    href_list = [item for sublist in href_list if sublist for item in sublist if item is not None]

                    if len(href_list)>0:
                        for charschool in href_list:
                            download_ca_s4(charschool,subject,cCounty,year,report=None,char=True,option=option)
                    
                except TimeoutException:
                    print("no chater Element may not clickable, please check",driver.current_url)
                    filepath=os.path.join(folder_path,cCounty+'.xlsx')
                    table_download(driver,filepath)
            else:
                filepath=os.path.join(folder_path,cCounty+'.xlsx') 
                table_download(driver,filepath)

        time.sleep(3)
        driver.get(step3)
            
    # Make sure all downaload have been finished
    time.sleep(3) 

In [ ]:
chrome_options = Options()
chrome_options.add_argument('--headless')
driver = webdriver.Chrome(executable_path='/Users/dai/Downloads/chromedriver_mac_arm64/chromedriver', options=chrome_options)
#driver = webdriver.Chrome(executable_path='/Users/dai/Downloads/chromedriver_mac_arm64/chromedriver')
missing_list=''

level='County'
for ind in df1.iloc[list(range(1,14))+[-2],:].index:
    subject=df1.loc[ind,'subject']
    years=df1.loc[ind,'exist_year']
    for year in years:
        for cCounty in cCountys:
            cds=cCounty.split('+')[0]
            try:
                if subject=='SpecEd':
                    step3='https://dq.cde.ca.gov/dataquest/SearchName.asp?rbTimeFrame=oneyear&rYear={}&cCounty={}&Topic={}&Level={}&submit1=Submit'.format(year,cCounty,subject,level) 
                    #https://dq.cde.ca.gov/dataquest/SearchName.asp?rbTimeFrame=oneyear&rYear=2018-19&cCounty=01+ALAMEDA&Topic=SpecEd&Level=County&submit1=Submit
                    download_ca_SpecEd(step3,subject,year,cCounty)  
                elif subject=='Foster':
                    reports=['fosterGrdEnrl','fosterGrdRace']
                    for report in reports:
                        step4='https://dq.cde.ca.gov/dataquest/{}/{}.aspx?level={}&cds={}&year={}'.format(subject,report,level,cds,year)
                        #https://dq.cde.ca.gov/dataquest/foster/fosterGrdEnrl.aspx?level=County&cds=01&year=2020-21
                        #https://dq.cde.ca.gov/dataquest/foster/fosterGrdRace.aspx?level=County&county=01&year=2020-21
                        download_ca_s4(step4,subject,cCounty,year,report)      
                elif subject =='Paif':
                    reports=['StfFteClassified','StfFteClassifiedLevels']
                    for report in reports:
                        step4='https://dq.cde.ca.gov/dataquest/dqcensus/{}.aspx?cds={}&agglevel={}&year={}'.format(report,cds,level,year)
                        #https://dq.cde.ca.gov/dataquest/dqcensus/StfFteClassified.aspx?cds=01&agglevel=County&year=2021-22
                        #https://dq.cde.ca.gov/dataquest/dqcensus/StfFteClassifiedLevels.aspx?cds=01&agglevel=County&year=2021-22
                        download_ca_s4(step4,subject,cCounty,year,report)
                elif subject=='FPRM':
                    step4='https://dq.cde.ca.gov/dataquest/Cbeds2.asp?FreeLunch=on&cChoice=CoProf2&cYear={}&TheCounty={}&cLevel=County&cTopic={}&myTimeFrame=S&submit1=Submit'.format(year,cCounty,subject)
                    #https://dq.cde.ca.gov/dataquest/Cbeds2.asp?FreeLunch=on&cChoice=CoProf2&cYear=2021-22&TheCounty=01%2CALAMEDA&cLevel=County&cTopic=FRPM&myTimeFrame=S&submit1=Submit
                    download_ca_s4(step4,subject,cCounty,year,report=None)
                elif subject=='Hires':
                    #https://dq.cde.ca.gov/dataquest/dqcensus/StfTchHires.aspx?cdcode=01&agglevel=County&year=2022-23
                    step4='https://dq.cde.ca.gov/dataquest/dqcensus/StfTchHires.aspx?cdcode={}&agglevel={}&year={}'.format(cds,level,year)
                    download_ca_s4(step4,subject,cCounty,year,report=None)
                else:    
                    step3='https://dq.cde.ca.gov/dataquest/SearchName.asp?rbTimeFrame=oneyear&rYear={}&cCounty={}&Topic={}&Level={}&submit1=Submit'.format(year,cCounty,subject,level) 
                    download_ca_t2(step3,subject,cCounty,year)
                time.sleep(2)    
            except:
                print(subject,year,cCounty)
                missing_list = missing_list + subject  +' '+ year  +' '+ cCounty+'\n'
                time.sleep(20) 
                driver = webdriver.Chrome(executable_path='/Users/dai/Downloads/chromedriver_mac_arm64/chromedriver', options=chrome_options)


LC 2017-18 02+ALPINE: the no-chater option of some county is not clickable

miss="""SpecEd 2018-19 02+ALPINE
SpecEd 2018-19 22+MARIPOSA
SpecEd 2017-18 02+ALPINE
SpecEd 2017-18 08+DEL+NORTE
SpecEd 2017-18 22+MARIPOSA"""

above page is not clickable

In [ ]:
driver = webdriver.Chrome(executable_path='/Users/dai/Downloads/chromedriver_mac_arm64/chromedriver')
#missing_list='LC 2017-18 02+ALPINE'

if missing_list!='':
    missing_list2=''
    level='County'
    for i in missing_list.split('\n'):
        subject= i.split(' ')[0]
        year= i.split(' ')[1]
        cCounty=i.split(' ')[2]
        cds=cCounty.split('+')[0]
        try:
            if subject=='SpecEd':
                step3='https://dq.cde.ca.gov/dataquest/SearchName.asp?rbTimeFrame=oneyear&rYear={}&cCounty={}&Topic={}&Level={}&submit1=Submit'.format(year,cCounty,subject,level) 
                #https://dq.cde.ca.gov/dataquest/SearchName.asp?rbTimeFrame=oneyear&rYear=2018-19&cCounty=01+ALAMEDA&Topic=SpecEd&Level=County&submit1=Submit
                download_ca_SpecEd(step3,subject,year,cCounty)  
            elif subject=='Foster':
                reports=['fosterGrdEnrl','fosterGrdRace']
                for report in reports:
                    step4='https://dq.cde.ca.gov/dataquest/{}/{}.aspx?level={}&cds={}&year={}'.format(subject,report,level,cds,year)
                    #https://dq.cde.ca.gov/dataquest/foster/fosterGrdEnrl.aspx?level=County&cds=01&year=2020-21
                    #https://dq.cde.ca.gov/dataquest/foster/fosterGrdRace.aspx?level=County&county=01&year=2020-21
                    download_ca_s4(step4,subject,cCounty,year,report)      
            elif subject =='Paif':
                reports=['StfFteClassified','StfFteClassifiedLevels']
                for report in reports:
                    step4='https://dq.cde.ca.gov/dataquest/dqcensus/{}.aspx?cds={}&agglevel={}&year={}'.format(report,cds,level,year)
                    #https://dq.cde.ca.gov/dataquest/dqcensus/StfFteClassified.aspx?cds=01&agglevel=County&year=2021-22
                    #https://dq.cde.ca.gov/dataquest/dqcensus/StfFteClassifiedLevels.aspx?cds=01&agglevel=County&year=2021-22
                    download_ca_s4(step4,subject,cCounty,year,report)
            elif subject=='FPRM':
                step4='https://dq.cde.ca.gov/dataquest/Cbeds2.asp?FreeLunch=on&cChoice=CoProf2&cYear={}&TheCounty={}&cLevel=County&cTopic={}&myTimeFrame=S&submit1=Submit'.format(year,cCounty,subject)
                #https://dq.cde.ca.gov/dataquest/Cbeds2.asp?FreeLunch=on&cChoice=CoProf2&cYear=2021-22&TheCounty=01%2CALAMEDA&cLevel=County&cTopic=FRPM&myTimeFrame=S&submit1=Submit
                download_ca_s4(step4,subject,cCounty,year,report=None)
            elif subject=='Hires':
                #https://dq.cde.ca.gov/dataquest/dqcensus/StfTchHires.aspx?cdcode=01&agglevel=County&year=2022-23
                step4='https://dq.cde.ca.gov/dataquest/dqcensus/StfTchHires.aspx?cdcode={}&agglevel={}&year={}'.format(cds,level,year)
                download_ca_s4(step4,subject,cCounty,year,report=None)
            else:    
                step3='https://dq.cde.ca.gov/dataquest/SearchName.asp?rbTimeFrame=oneyear&rYear={}&cCounty={}&Topic={}&Level={}&submit1=Submit'.format(year,cCounty,subject,level) 
                download_ca_t2(step3,subject,cCounty,year)
            time.sleep(2)  
        except:
            print(subject,year,cCounty)
            missing_list2 = missing_list2 + subject  +' '+ year  +' '+ cCounty+'\n'
            time.sleep(20) 
            driver = webdriver.Chrome(executable_path='/Users/dai/Downloads/chromedriver_mac_arm64/chromedriver')         
    print(missing_list2)
    missing_list=missing_list2

# Combine data from aws and local download

In [ ]:
import os
import shutil

dir1 = '/Users/dai/Desktop/enrollment combine/Enrollment_aws'
dir2 = '/Users/dai/Desktop/enrollment combine/Enrollment_local'

output_dir= '/Users/dai/Desktop/enrollment combine/all'

for root, dirs, files in os.walk(dir1):
    for file in files:
        if not file.startswith('.'):  
            path = os.path.join(root, file)
            relative_path = os.path.relpath(path, dir1)  
            output_path = os.path.join(output_dir, relative_path)
            os.makedirs(os.path.dirname(output_path), exist_ok=True)
            shutil.copyfile(path, output_path)

for root, dirs, files in os.walk(dir2):
    for file in files:
        if not file.startswith('.'): 
            path = os.path.join(root, file)
            relative_path = os.path.relpath(path, dir2) 
            output_path = os.path.join(output_dir, relative_path)
            os.makedirs(os.path.dirname(output_path), exist_ok=True)
            shutil.copyfile(path, output_path)

In [18]:
import os
import pandas as pd


data = []
for root, dirs, files in os.walk('/Users/dai/Desktop/Capstone-AllianceBernstein/CA_data/Enrollment'):
    for file in files:
        path=os.path.join(root, file)
        if '.DS_Store' in path:
            continue
        name=path.replace('/Users/dai/Desktop/Capstone-AllianceBernstein/CA_data/Enrollment','').split('/')
        #print(name)
        report=name[1]
        year=name[2]
        cCounty_num=name[3].split('+')[0]
        data.append([report, year, cCounty_num])
df = pd.DataFrame(data, columns=['Report', 'Year', 'County'])
df=df.sort_values(['Report', 'Year', 'County'])
df=df.drop_duplicates()
df=df.groupby(['Report','Year'])['County'].apply(lambda x: ','.join(x)).reset_index()

df['County'] = df['County'].apply(lambda x: [int(num) for num in x.split(',') if num])
full_set = set(range(1, 59))

for ind,lst in enumerate(df['County']):
    num_set = set(lst)
    missing_set = full_set - num_set
    df.loc[ind,'miss_num']=','.join(map(str, sorted(list(missing_set))))
df=df[df['miss_num']!='']
df.loc[:,'miss_county']=df.loc[:,'miss_num'].apply(lambda x: ','.join([cCountys[int(val)-1] for val in x.split(',')]))

# get unique part.
df['County']=df['County'].apply(tuple)
df=df.iloc[:,1:].drop_duplicates()

,Report,Year,County,miss_num,miss_county
0,ELAS,2017-18,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","35,36,37,38,39","35+SAN+BENITO,36+SAN+BERNARDINO,37+SAN+DIEGO,3..."
5,ELASLevels,2017-18,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","35,36,37,38,39","35+SAN+BENITO,36+SAN+BERNARDINO,37+SAN+DIEGO,3..."
10,EnrAgeGr,2017-18,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","36,37,38,39","36+SAN+BERNARDINO,37+SAN+DIEGO,38+SAN+FRANCISC..."
15,EnrAgeLvl,2017-18,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","36,37,38,39","36+SAN+BERNARDINO,37+SAN+DIEGO,38+SAN+FRANCISC..."
20,EnrChrEth,2017-18,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","36,37,38,39","36+SAN+BERNARDINO,37+SAN+DIEGO,38+SAN+FRANCISC..."
25,EnrChrLvl,2017-18,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","36,37,38,39","36+SAN+BERNARDINO,37+SAN+DIEGO,38+SAN+FRANCISC..."
30,EnrChrSub,2017-18,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","36,37,38,39","36+SAN+BERNARDINO,37+SAN+DIEGO,38+SAN+FRANCISC..."
35,EnrChrYr,2017-18,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","36,37,38,39","36+SAN+BERNARDINO,37+SAN+DIEGO,38+SAN+FRANCISC..."
40,EnrEth2,2017-18,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","36,37,38,39","36+SAN+BERNARDINO,37+SAN+DIEGO,38+SAN+FRANCISC..."
45,EnrEthGr2,2017-18,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","36,37,38,39","36+SAN+BERNARDINO,37+SAN+DIEGO,38+SAN+FRANCISC..."


In [22]:
driver = webdriver.Chrome(executable_path='/Users/dai/Downloads/chromedriver_mac_arm64/chromedriver')
level='County'
subject='Enrollment'
year='2017-18'
for cCounty in '35+SAN+BENITO,36+SAN+BERNARDINO,37+SAN+DIEGO,38+SAN+FRANCISCO,39+SAN+JOAQUIN'.split(','):
    cds=cCounty.split('+')[0]
    step3='https://dq.cde.ca.gov/dataquest/SearchName.asp?rbTimeFrame=oneyear&rYear={}&cCounty={}&Topic={}&Level={}&submit1=Submit'.format(year,cCounty,subject,level) 
    download_ca_t2(step3,subject,cCounty,year) 
    time.sleep(3)  

/var/folders/3c/40ljyfqs6z97lnyy1drkzt9w0000gn/T/ipykernel_70790/2461493667.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path='/Users/dai/Downloads/chromedriver_mac_arm64/chromedriver')
